# Project 2: Using Multidimensional Data with Gramfort's Approach

In [2]:
# graphical libraries
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
from IPython.display import Image
from IPython.display import display
plt.style.use('seaborn-white')

In [18]:
# computational libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats 
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay

In [19]:
lm = LinearRegression()
scale = StandardScaler()
qscale = QuantileTransformer()

## Alternate Lowess Function
This is an alternate SciKit-Learn Compliant class that includes a lowess function we used in class previously, along with a Gaussian kernel and weights matrix function.

In [47]:
def Gaussian(x):
  if len(x.shape)==1:
    d = np.abs(x)
  else:
    # here, we are treating x as the difference between points
    d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*d**2))

def Tricubic(x):
  if len(x.shape)==1:
    d = np.abs(x)
  else:
    d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,70/81*(1-d**3)**3)

def kernel_function(xi,x0,kern, tau): 
    return kern((xi - x0)/(2*tau))

def weights_matrix(x,x_new,kern,tau):
  if np.isscalar(x_new):
    return kernel_function(x,x_new,kern,tau)
  else:
    n = len(x_new)
    return np.array([kernel_function(x,x_new[i],kern,tau) for i in range(n)])

class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau
    
    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_

        w = weights_matrix(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w).dot(x.reshape(-1,1)),np.diag(w).dot(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        elif len(x.shape)==1:
          n = len(x_new)
          yest_test = np.zeros(n)
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[i,:]).dot(x.reshape(-1,1)),np.diag(w[i,:]).dot(y.reshape(-1,1)))
            yest_test[i] = lm.predict(x_new[i].reshape(-1,1))
        else:
          n = len(x_new)
          yest_test = np.zeros(n)
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[i,:]).dot(x),np.diag(w[i,:]).dot(y.reshape(-1,1)))
            yest_test[i] = lm.predict(x_new[i].reshape(1,-1))
        return yest_test

## Original Gramfort Function
Here is the one dimensional version from class that does not work for multidimensional data.

In [4]:
def lowess_ag(x, y, f=2. / 3., iter=3):
    """lowess(x, y, f=2./3., iter=3) -> yest
    Lowess smoother: Robust locally weighted regression.
    The lowess function fits a nonparametric regression curve to a scatterplot.
    The arrays x and y contain an equal number of elements; each pair
    (x[i], y[i]) defines a data point in the scatterplot. The function returns
    the estimated (smooth) values of y.
    The smoothing span is given by f. A larger value for f will result in a
    smoother curve. The number of robustifying iterations is given by iter. The
    function will run faster with a smaller number of iterations.
    """
    n = len(x)
    r = int(ceil(f * n))
    h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
    w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)
    w = (1 - w ** 3) ** 3
    yest = np.zeros(n)
    delta = np.ones(n)
    for iteration in range(iter):
        for i in range(n):
            weights = delta * w[:, i]
            b = np.array([np.sum(weights * y), np.sum(weights * y * x)])
            A = np.array([[np.sum(weights), np.sum(weights * x)],
                          [np.sum(weights * x), np.sum(weights * x * x)]])
            beta = linalg.solve(A, b)
            yest[i] = beta[0] + beta[1] * x[i]

        residuals = y - yest
        s = np.median(np.abs(residuals))
        delta = np.clip(residuals / (6.0 * s), -1, 1)
        delta = (1 - delta ** 2) ** 2

    return yest

## New Multidimensional Function
This distance function is useful for computing the Euclidean distance.

In [44]:
def dist(u,v):
  if len(v.shape)==1:
    # this reshapes into a row vector
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

Finally, we have our Multidimensional version of the Gramfort approach.

In [28]:
def lowess_ag_md(x, y, xnew, f=2/3, iter=3, intercept=True, kernel='tricubic'):

  n = len(x)

  # calculating the amount of neighbors we are considering
  r = int(ceil(f * n))

  # one dimensional y and x become column vectors here
  if len(y.shape)==1:
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  # we need to add a column of ones to account for the intercept
  # this is by default, however the user has the option to not 
  # include this step
  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  # calculating the Euclidean distance for each observation
  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]

  # we now clip any distance that falls outside of our bounds
  w = np.clip(dist(x,x) / h, 0.0, 1.0)

  # we apply our kernel to get the weights, giving closer neighbors more weight
  # by default, we use tricubic, however, the user may specify an alternate
  if kernel in {'Quartic','quartic','q'}:
    w = (1 - w ** 2) ** 2
  elif kernel in {'Epanechnikov','epanechnikov','ep'}:
    w = (1 - w ** 2)
  elif kernel in {'Exponential','exponential','ex','exp','Gaussian','gaussian','g'}:
    w = np.exp(-1/2*w**2)
  else:
    w = (1 - w ** 3) ** 3

  # initialize our y estimate and delta
  yest = np.zeros(n)
  delta = np.ones(n)

  for iteration in range(iter):
    for i in range(n):
      # convert our weights to a diagonal matrix
      W = np.diag(w[:,i])

      # this is our ordinate values
      b = np.transpose(x1).dot(W).dot(y)

      # this is our coefficient matrix
      A = np.transpose(x1).dot(W).dot(x1)

      # L2 regularization, this ensures that our matrix equation can always be solved
      A = A + 0.0001*np.eye(x1.shape[1])

      # finally, we solve our matrix equation
      beta = linalg.solve(A, b)
      
      # create the y estimate
      yest[i] = np.dot(x1[i],beta)

    # now we get the residuals and calculate our cutoff for outliers
    residuals = y - yest

    # quartiles and IQR
    Q1 = np.percentile(residuals,25)
    Q3 = np.percentile(residuals,75)
    IQR = Q3-Q1

    # upper and lower bound for outliers
    upper = Q3 + (1.5 * IQR)
    lower = Q1 = (1.5 * IQR)

    # these values help us standardize the data to have the lower bound be -1
    # and the upper bound be 1
    mid = (upper + lower) / 2
    div = upper-mid

    # finally, we clip all outliers
    delta = np.clip((residuals - mid)/div, -1, 1)
    delta = (1 - delta ** 2) ** 2

  # this is how we handle training and testing data (x, xnew)
  if x.shape[1]==1:
    # this is a one dimensional interpolator
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    # otherwise, we need to use principal component analysis on our higher dimensional data
    # this is because there may be relationships between some of the features
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ')
      f = LinearNDInterpolator(tri,y[ind])
      output[i] = f(pca.transform(xnew[i].reshape(1,-1))) # the output may have NaN's where the data points from xnew are outside the convex hull of X

  # the output may have NaN's where the data points from xnew are outside the convex hull of X
  # here, we will resolve that with nearest neighbor interpolation in higher dimensions
  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,y.ravel()) 
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

By creating this wrapper class below, we can have our above function be SciKitLearn-compliant.

In [39]:
class Lowess_AG_MD:
    def __init__(self, f = 1/10, iter = 3,intercept=True, kernel='tricubic'):
        self.f = f
        self.iter = iter
        self.intercept = intercept
        self.kernel = kernel
    
    def fit(self, x, y):
        f = self.f
        iter = self.iter
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        f = self.f
        iter = self.iter
        intercept = self.intercept
        kernel = self.kernel
        return lowess_ag_md(x, y, x_new, f, iter, intercept, kernel)

    def get_params(self, deep=True):
        return {"f": self.f, "iter": self.iter,"intercept":self.intercept,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

## Application
We will be applying our multidimensional Gramfort function to some testing data. In addition, we will use K-Fold Cross Validation and Grid Search for validating our results and optimizing hyperparameters.

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/cars.csv')

In [42]:
x = data[['CYL','ENG','WGT']].values
y = data['MPG'].values

We will compare our results with the results of a Random Forest Regressor and with the alternate Lowess Function.
### K-Fold Cross Validation

In [48]:
mse_lw_ag = []
mse_lw = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=5)
model_lw = Lowess(kernel=Tricubic,tau=0.02)
model_lw_ag = Lowess_AG_MD(f=1/65,iter=3,intercept=True,kernel='tricubic')

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_lw_ag.fit(xtrain,ytrain)
  yhat_lw_ag = model_lw_ag.predict(xtest)

  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)
  
  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lw_ag.append(mse(ytest,yhat_lw_ag))
  mse_lw.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print("The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lw_ag)))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lw)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 26.089907926313504
The Cross-validated Mean Squared Error for Locally Weighted Regression is : 1.6841558606802643e+19
The Cross-validated Mean Squared Error for Random Forest is : 17.206807368339287


### Grid Search CV

In [49]:
# here is our pipeline for grid search
lwr_pipe = Pipeline([('zscores', StandardScaler()),
                     ('lwr', Lowess_AG_MD())])

In [50]:
# now we specify which parameters we want to consider
params = [{'lwr__f': [1/i for i in range(3,15)],
         'lwr__iter': [1,2,3,4],
         'lwr__kernel': ['Tricubic','Quartic','Epanechnikov','Exponential']}]

In [51]:
# finally we perform the grid search
gs_lowess = GridSearchCV(lwr_pipe,
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)
gs_lowess.fit(x, y)
gs_lowess.best_params_

{'lwr__f': 0.3333333333333333, 'lwr__iter': 1, 'lwr__kernel': 'Tricubic'}

In [52]:
# and we look at our score
gs_lowess.score(x,y)

-1.6547449275510306

As we can see, our best combination of parameters was f=1/3, iter=1, and kernel=Tricubic